In [1]:
import os
import sys
import psycopg2
from sqlalchemy import create_engine
from io import StringIO
import pandas as pd
import csv
up_one =  (os.path.abspath(os.path.join(os.getcwd(), os.pardir)))
sys.path.append(up_one)
up_two =  (os.path.abspath(os.path.join(os.getcwd(), os.pardir, os.pardir)))
sys.path.append(up_two)
from minorlab import metapdb
import subprocess

import pprint
pp = pprint.PrettyPrinter(indent=4)

In [26]:
#Project specific functions
# Make blast dbs
def makeblastdb (fasta):
    #  Note : This was problematic on OneDrive 
    root = fasta.replace('.fa','')
    cmd = f"""{makedb} -in {fasta} -out {root} -dbtype prot -title "Blastdb for {fasta}" -parse_seqids """
    print (cmd)
    db=subprocess.getoutput(cmd)
    if db[0:5]=="USAGE":
        print (f"Something went wrong with creation of the database\n{db}")

In [3]:
conn={}  # A dict of connections
cursor={}    # A dict of cursors
engine={}
def connect_db(db):
    print (f"     Connecting to {db['name']}...")
    try:
        #print (f"""    con_string is { db['con_string'] }""")
        exec (f"""conn[ "{db["name"]}" ] = psycopg2.connect("{db["con_string"]}")""" )
        exec (f"""cursor[ "{db['name']}" ] = conn[ "{db['name']}" ].cursor()""") # Pandas read_sql_query does not need a cursor
        exec (f"""engine["{db['name']}"] = create_engine( "{db['engine_string']}") """)
        print(f"  Connected to { db['name'] }.")
    except:
        print ('Check VPN. Not Connected.   <<<<===========#################')
connect_db(metapdb)
#connect_db(manuscripts)
print ('Connections done.')
# Run this cell to use single database functions
con = conn['metapdb']
cur = cursor['metapdb']
eng = engine['metapdb']

def df_to_empty_table(df, table,db='metapdb'):
    #create table using SLQalchemy 
    parts = table.split('.')
    if len(parts) ==1:
        schema='public'
        table_name=table
    else:
        schema, table_name = parts
    empty_df=  df.loc[df.iloc[:,0]=='Nonsense']
    try:
        empty_df.to_sql( schema=schema,name=table_name, con=engine[db], if_exists = 'replace', index=False)
        print (f'  Empty table {table} created for DataFrame with columns {empty_df.columns.to_list()}')
    except Exception as error:
        print (f'There was a problem with the SQLalchemy part. No table created.\n{error}')
        return 0
    return 1
    
def df_to_table(df, table,db='metapdb'):
    ''' Loads a DataFrame into a table. 
        If table is in the form schema.table, the schema will be used, otherwise schema="public".
        3rd ARG is databse; metapdb is default.'''
    if df_to_empty_table(df,table,db) == 0:
        return 0
    # save dataframe to an in memory buffer
    buffer = StringIO()
    
    df.to_csv(buffer, index=False, header=False,sep=';',quoting=csv.QUOTE_NONE,escapechar='\\')
    buffer.seek(0)
    
    # Try to copy from the buffer to the table
    try:
        print (f"""  Starting copy_from(buffer)""" )
        cursor[db].copy_from(buffer, table, sep=";",)
        conn[db].commit()
    except (Exception, psycopg2.DatabaseError) as error:
        print("Error: %s" % error)
        conn[db].rollback()
        return 1
    print(f"df_to_table done for {table}.") 

     Connecting to metapdb...
  Connected to metapdb.
Connections done.


## Slight reformat of gencode information

In [79]:
with open ('Data/gencode.v43.pc_translations.fa', 'r') as f_in, open ('Data/gencode_v43_translations_mod.fa', 'w') as f_out:
    for line in f_in.readlines():
        if line[0] == ">":
            parts = line.split('|')
            f_out.write(f"{parts[0]}|{parts[5]}\n")
        else:
            f_out.write(line)

### Get list of human pdbs directly from PDB


In [ ]:
human_pdb= pd.read_csv ('Data/PDBe_human_entities.csv')
human_pdb_red = human_pdb.loc[:,['pdb_id','entity_id']]
df_to_table(human_pdb_red, 'temp_dcoop.human_pdb')

In [4]:
sql = """select ep.pdbid, ep.pdbx_strand_id as chains
            , ep.pdbx_seq_one_letter_code_can  as seq
        from temp_dcoop.human_pdb hp
        join pdbj.entity_poly ep on ep.pdbid = hp.pdb_id
            and hp.entity_id = cast(ep.entity_id as INT)
        where ep.type='polypeptide(L)'
        order by pdbid, pdbx_strand_id"""
df = pd.read_sql (sql,eng)
df
len (df['pdbid'].unique())

61321

## Write out as pdb_human_seqs.fa

In [ ]:
with open ('Data/pdb_human_seqs-230227.fa', 'w') as f:
    for i,parts in df.iterrows():
        chains = chains.replace(',','_')
        f.write(f">{parts['pdbid']}_{parts['chains']}\n{parts['seq']}\n")

## eliminate seqs < 30 AA and group by 100% identity

Group by reduces the list from ~97K to ~39K  
Removing chains less than 30AA removes 3167 


In [5]:
df2=df.copy()
df2['id']=df2['pdbid']+'_'+ df2['chains']
df2=df2.loc[:,['id','seq']]
df2 = df2.loc[ df2['seq'].str.len() >= 30 ]
print (f'df({len(df)}) - df2({len(df2)}) = {len(df) -  len(df2)} removed by 30 AA cutoff') 
df2.head()

df(96902) - df2(91217) = 5685 removed by 30 AA cutoff


,id,seq
0,"10gs_A,B",PPYTVVYFPVRGRCAALRMLLADQGQSWKEEVVTVETWQEGSLKAS...
1,"11gs_A,B",MPPYTVVYFPVRGRCAALRMLLADQGQSWKEEVVTVETWQEGSLKA...
2,121p_A,MTEYKLVVVGAGGVGKSALTIQLIQNHFVDEYDPTIEDSYRKQVVI...
3,12ca_A,MSHHWGYGKHNGPEHWHKDFPIAKGERQSPVDIDTHTAKYDPSLKP...
4,"12gs_A,B",MPPYTVVYFPVRGRCAALRMLLADQGQSWKEEVVTVETWQEGSLKA...


In [6]:
groups = df2.groupby('seq')['id'].apply(list)
df2_g = groups.reset_index(name='chains')
print (f'df2({len(df2)}) - df2_g({len(df2_g)}) = {len(df2) -  len(df2_g)} less seqs after clustering at 100%') 
df2_g.head()

df2(91217) - df2_g(36104) = 55113 less seqs after clustering at 100%


,seq,chains
0,AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA,[6zm5_v]
1,AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA,[6g4s_k]
2,AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA...,"[6zp4_X, 6zvj_Y, 7a09_X]"
3,AAAAAAAAAAAAAMKTIIALSYIFCLVFADYKDDDDKEKGISVPDH...,[7evw_R]
4,AAAAAAMIVQRVVLNSRPGKNGNPVAENFRMEEVYLPDNINEGQVQ...,"[2zb4_A, 2zb7_A, 2zb8_A]"


### get representative chain as for name and write out .fa

In [ ]:
df2_g['rep'] = [i[0] for i in df2_g['chains']]
with open ('Data/pdb_unique_human_seqs_230301.fa', 'w') as f:
    for i,parts in df2_g.iterrows():
        f.write(f">{parts['rep']}\n{parts['seq']}\n")

# Blast part
blastp -db ./a_gencode_fasta_same_prot_clustered.v35.fa -query ./uniprot_reviewed_canonical_and_isoform.fasta  
      -outfmt "6 qseqid qlen qstart qend sseqid slen sstart send evalue bitscore length nident gaps"   
      -out run_UN_to_GC_blast_result.tsv -num_threads 24 >> run_UN_to_GC_blast_result_stdout.txt 2>&1
makeblastdb -in <fasta file> -out <database name> -dbtype <type> -title <title> -parse_seqids
    
__[TEXT](https://www.ncbi.nlm.nih.gov/books/NBK279684/table/appendices.T.options_common_to_all_blast/)__

In [99]:
blast_path = "/home/daroc/ncbi-blast-2.13.0+/bin"
blastp_ = os.path.join (blast_path, 'blastp')
makedb = os.path.join (blast_path, 'makeblastdb')
s='gencode_v43_translations_mod.fa'
#q='one_pdb.fa'
q='some_pdbs.fa'
blast_options="-evalue 1e-20 -max_hsps 2"
columns =  ['qseqid','qlen', 'qstart', 'qend', 'sseqid', 'slen', 'sstart', 'send', 'evalue', 'bitscore', 'length', 'nident', 'gaps','gapopen', 'mismatch']


#  Should not need to edit below here 
makeblastdb(q)
s_root = s.replace('.fa','')
column_str = " ".join(columns)
outfmt = f"6 {column_str}"


/home/daroc/ncbi-blast-2.13.0+/bin/makeblastdb -in some_pdbs.fa -out some_pdbs -dbtype prot -title "Blastdb for some_pdbs.fa" -parse_seqids 


In [100]:
# DF output 
blast_cmd = f'{blastp} -query "Data/{q}" -subject "Data/{s}" {blast_options} -outfmt "{outfmt}"'
print (f"{blast_cmd}\n")
blast_results=subprocess.getoutput(blast_cmd)
results_list = [line.split('\t') for line in blast_results.splitlines()]
blast_df = pd.DataFrame(results_list, columns = columns)
try:
    blast_df.to_excel('Data/some_all.xlsx')
except:
    print ("\n\nCouldn't write Excel file!!!")
blast_df


/home/daroc/ncbi-blast-2.13.0+/bin/blastp -query "Data/some_pdbs.fa" -subject "Data/gencode_v43_translations_mod.fa" -evalue 1e-20 -max_hsps 2 -outfmt "6 qseqid qlen qstart qend sseqid slen sstart send evalue bitscore length nident gaps gapopen mismatch"



,qseqid,qlen,qstart,qend,sseqid,slen,sstart,send,evalue,bitscore,length,nident,gaps,gapopen,mismatch
0,7evw_R,599,38,574,ENSP00000286201.1|FZD7-201,574,38,574,0.0,1115,537,537,0,0,0
1,7evw_R,599,36,574,ENSP00000287934.2|FZD1-201,647,103,647,0.0,905,551,440,18,6,93
2,7evw_R,599,38,574,ENSP00000323901.3|FZD2-201,565,28,565,0.0,893,553,434,31,3,88
3,7evw_R,599,49,563,ENSP00000354607.3|FZD5-201,585,33,536,2.42e-180,523,533,276,47,14,210
4,7evw_R,599,44,589,ENSP00000229030.4|FZD10-201,581,29,573,1.48e-165,485,562,261,33,10,268
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2989,1iat_A,557,1,106,ENSP00000466191.1|GPI-217,146,41,146,6.42e-70,222,106,106,0,0,0
2990,1iat_A,557,1,105,ENSP00000496041.1|GPI-218,145,41,145,3.28e-69,220,105,105,0,0,0
2991,1iat_A,557,1,99,ENSP00000468298.2|GPI-206,139,41,139,5.19e-65,209,99,99,0,0,0
2992,1iat_A,557,1,76,ENSP00000464797.2|GPI-207,116,41,116,3.21e-47,162,76,76,0,0,0


In [101]:
# For full results
blast_cmd = f'{blastp} -query "Data/{q}" -subject "Data/{s}"  {blast_options}'
print (f"{blast_cmd}\n")
blast_results=subprocess.getoutput(blast_cmd)
print (blast_results)

/home/daroc/ncbi-blast-2.13.0+/bin/blastp -query "Data/some_pdbs.fa" -subject "Data/gencode_v43_translations_mod.fa"  -evalue 1e-20 -max_hsps 2



IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [47]:

def classify_matches(df):
    insertion= 0
    deletion= 0
    mutation=0
    for i,row in df.iterrows():
        #slens equal
        if r['mismatch'] >= 1:
            mutation=1
        
        
classify_matches(blast_df)

0
1
2
3
4
5
6
7
8
9
10
